# Data Sampling

## Setup

In [12]:
import sys

!{sys.executable} -m pip install --quiet --user --upgrade pandas==1.*
!{sys.executable} -m pip install --quiet --user --upgrade -r requirements.txt

In [13]:
from goodies import *

## Data Collection

In [14]:
from dcollect import plugins

modules = {'http': plugins.fasthttp()}
headers = None

### YouTube (United States)

Initial setup. Be sure to have your API key ready. For details on how to obtain an API key, read [YouTube Data API Overview, Introduction: Before you start](https://developers.google.com/youtube/v3/getting-started#before-you-start).

In [15]:
from dcollect import api_youtube as youtube
from dcollect import api_youtubei as youtubei

# This key is for testing ONLY. DO NOT release to the public!
api_key_testing = None
api_key = api_key_testing or input('YouTube Data API Key: ')

#### Search

In [16]:
count = 200
keyword = ''

##### STEP 1  API Object Creation

In [17]:
# create a YouTube API object
youtube_o = youtube.api(
    modules = modules,
    headers = headers,
    key = api_key
)

# create a YouTube Internals API object
youtubei_o = youtubei.api(
    modules = modules,
    headers = headers
)

##### STEP 2  Data Collection

In [18]:
from dcollect.utils.thread import threading, thread
from dcollect.utils.log import log

# set logging level
log.enable(level = log.levels.WARNING)


df_trending = df_from_json(
    youtube_o.video.trending(
        count = count
    )
)

df_channels = None
df_ads = None

thread.start([
    threading.Thread(
        # - channels
        target = lambda: \
            globals().update(
                df_channels = df_from_json(
                    youtube_o.channel.info(
                        id = df_trending['creator.id']
                    )
                )
            )
    ),
    threading.Thread(
        # - ad placements
        target = lambda: \
            globals().update(
                df_ads = df_from_json(
                    youtubei_o.ad.placements(
                        id = df_trending['id']
                    )
                )
            )
    )
])
thread.join()

##### STEP 3  Data Cleaning


In [19]:
# - ads (filter)
def filter_has_ad(ads):
    return not ads == None 
def filter_has_ad_beginning(ads):
    if ads == None:
        return False
    for ad in ads:
        if ad['kind'] == youtubei.resource.ad.kinds.START:
            return True
    return False
# - * (filter)
def drop_common(df, df_other, *args, **kwargs):
    return df.drop(columns = df.columns & df_other.columns, *args, **kwargs)

# - search
df_trending.set_index(['id', 'creator.id'], inplace = True)
# - channels
df_channels = df_channels.add_prefix('creator.')
df_channels.set_index(['creator.id'], inplace = True)
# - ads
df_ads.set_index(['id'], inplace = True)
df_ads['has_ad'] = df_ads['ads'].apply(filter_has_ad)
df_ads['has_ad_at_beginning'] = \
        df_ads['ads'].apply(filter_has_ad_beginning)
df_ads.drop('ads', axis = 'columns', inplace = True)

In [20]:
# - search (with details)
df_trending_details = df_trending.copy()
# - channels
df_trending_details = df_trending_details.merge(
    df_channels, 
    right_index = True, 
    left_on = 'creator.id', 
    copy = False
)
# - ads
df_trending_details = df_trending_details.merge(
    df_ads, 
    right_index = True, 
    left_on = 'id', 
    copy = False
)

##### STEP 4  Data Inspection


In [21]:
# take a brief look at our data
df_report(df_trending_details, name = 'Trending')

---

# Results - Trending

## Data Preview

title  \
id          creator.id                                                                    
tRMGdWValIk UClQubH2NeMmGLTLgNdLBwXg          Last To Stop Customizing Wins Lamborghini   
            UClQubH2NeMmGLTLgNdLBwXg          Last To Stop Customizing Wins Lamborghini   
            UClQubH2NeMmGLTLgNdLBwXg          Last To Stop Customizing Wins Lamborghini   
            UClQubH2NeMmGLTLgNdLBwXg          Last To Stop Customizing Wins Lamborghini   
28RxpkcVAFw UClQubH2NeMmGLTLgNdLBwXg  I Customized A Nintendo Switch In 10 Hours, 1 ...   

                                                                            description  \
id          creator.id                                                                    
tRMGdWValIk UClQubH2NeMmGLTLgNdLBwXg  I can't believe we painted 6 Lamborghinis then...   
            UClQubH2NeMmGLTLgNdLBwXg  I can't believe we painted 6 Lamborghinis then...   
            UClQubH2NeMmGLTLgNdLBwXg  I can't believe we painted 6 Lamborghinis then...   
            UClQubH2NeMmGLTLgNdLBwXg  I can't believe we painted 6 Lamborghinis then...   
28RxpkcVAFw UClQubH2NeMmGLTLgNdLBwXg  I can't believe we customized Nintendo switche...   

                                                          time  \
id          creator.id                                           
tRMGdWValIk UClQubH2NeMmGLTLgNdLBwXg 2021-03-08 03:54:59+00:00   
            UClQubH2NeMmGLTLgNdLBwXg 2021-03-08 03:54:59+00:00   
            UClQubH2NeMmGLTLgNdLBwXg 2021-03-08 03:54:59+00:00   
            UClQubH2NeMmGLTLgNdLBwXg 2021-03-08 03:54:59+00:00   
28RxpkcVAFw UClQubH2NeMmGLTLgNdLBwXg 2021-02-28 20:59:51+00:00   

                                              length  \
id          creator.id                                 
tRMGdWValIk UClQubH2NeMmGLTLgNdLBwXg 0 days 00:18:04   
            UClQubH2NeMmGLTLgNdLBwXg 0 days 00:18:04   
            UClQubH2NeMmGLTLgNdLBwXg 0 days 00:18:04   
            UClQubH2NeMmGLTLgNdLBwXg 0 days 00:18:04   
28RxpkcVAFw UClQubH2NeMmGLTLgNdLBwXg 0 days 00:19:19   

                                                                                   tags  \
id          creator.id                                                                    
tRMGdWValIk UClQubH2NeMmGLTLgNdLBwXg  [lamborghini, lambo, last to, last to stop, zh...   
            UClQubH2NeMmGLTLgNdLBwXg  [lamborghini, lambo, last to, last to stop, zh...   
            UClQubH2NeMmGLTLgNdLBwXg  [lamborghini, lambo, last to, last to stop, zh...   
            UClQubH2NeMmGLTLgNdLBwXg  [lamborghini, lambo, last to, last to stop, zh...   
28RxpkcVAFw UClQubH2NeMmGLTLgNdLBwXg           [nintendo switch, nintendo, zhc, gaming]   

                                           category  stats.like  \
id          creator.id                                            
tRMGdWValIk UClQubH2NeMmGLTLgNdLBwXg  Entertainment      214122   
            UClQubH2NeMmGLTLgNdLBwXg  Entertainment      214122   
            UClQubH2NeMmGLTLgNdLBwXg  Entertainment      214122   
            UClQubH2NeMmGLTLgNdLBwXg  Entertainment      214122   
28RxpkcVAFw UClQubH2NeMmGLTLgNdLBwXg  Entertainment      212929   

                                      stats.dislike  stats.comment  \
id          creator.id                                               
tRMGdWValIk UClQubH2NeMmGLTLgNdLBwXg           4698        16697.0   
            UClQubH2NeMmGLTLgNdLBwXg           4698        16697.0   
            UClQubH2NeMmGLTLgNdLBwXg           4698        16697.0   
            UClQubH2NeMmGLTLgNdLBwXg           4698        16697.0   
28RxpkcVAFw UClQubH2NeMmGLTLgNdLBwXg           4802        25062.0   

                                      stats.view video.quality creator.title  \
id          creator.id                                                         
tRMGdWValIk UClQubH2NeMmGLTLgNdLBwXg     4668939            HD           ZHC   
            UClQubH2NeMmGLTLgNdLBwXg     4668939            HD           ZHC   
            UClQubH2NeMmGLTLgNd

## Stats

,length,stats.like,stats.dislike,stats.comment,stats.view,creator.stats.follower,creator.stats.view,creator.stats.post
count,236,236.000000,236.000000,232.000000,2.360000e+02,2.360000e+02,2.360000e+02,236.000000
mean,0 days 00:11:22.796610169,138912.711864,4742.406780,13953.517241,2.786653e+06,5.140919e+06,1.675153e+09,4601.813559
std,0 days 00:10:26.050090570,195669.403670,18477.009132,30602.136098,3.641300e+06,6.118493e+06,2.520460e+09,11206.261368
min,0 days 00:00:23,1256.000000,164.000000,496.000000,1.971850e+05,1.100000e+04,1.220866e+07,23.000000
25%,0 days 00:03:22,16331.000000,354.000000,2222.000000,6.829160e+05,8.780000e+05,1.482879e+08,219.000000
50%,0 days 00:08:21,59930.000000,697.000000,4460.500000,1.411170e+06,1.880000e+06,6.903383e+08,477.000000
75%,0 days 00:16:22,162743.000000,3376.000000,10951.000000,3.058373e+06,8.370000e+06,2.063802e+09,1798.000000
max,0 days 00:49:23,834394.000000,142630.000000,182949.000000,1.837411e+07,2.660000e+07,1.393113e+10,49574.000000


---

##### STEP 5  Data Archiving (Cumulative)


In [22]:
import os

pickle_proto = 3
pickle_fname = 'dsamples/youtube_trending.pkl'

df_trending_details_final = None

if os.path.isfile(pickle_fname):
    df_trending_details_final = pd.read_pickle(pickle_fname)
    df_trending_details_final = df_trending_details.combine_first(
        other = df_trending_details_final
    )
else:
    df_trending_details_final = df_trending_details

df_trending_details_final.to_pickle(pickle_fname, protocol = pickle_proto)

# verify that we saved the correct data
df_trending_details_verify = pd.read_pickle(pickle_fname)
df_report(df_trending_details_verify, name = 'Trending (Verification)')

---

# Results - Trending (Verification)

## Data Preview

,,title,description,time,length,tags,category,stats.like,stats.dislike,stats.comment,stats.view,video.quality,creator.title,creator.description,creator.time,creator.stats.follower,creator.stats.view,creator.stats.post,has_ad,has_ad_at_beginning
id,creator.id,,,,,,,,,,,,,,,,,,,
CkTVoLamPio,UC-SJ6nODDmufqBzPBwCvYvQ,"Meghan, Duchess of Sussex, opens up about her ...","For the first time Meghan, Duchess of Sussex, ...",2021-03-08 14:01:55+00:00,0 days 00:10:19,"[cbs this morning, duchess of sussex, Meghan M...",News & Politics,32015,5461,7184.0,3461395,HD,CBS This Morning,"Each weekday morning, Gayle King, Anthony Maso...",2013-05-23 10:59:52+00:00,1880000,1455944229,29384,True,True
auOrYTASVqQ,UC-SJ6nODDmufqBzPBwCvYvQ,Oprah Winfrey on her bombshell Harry and Megha...,"Oprah Winfrey joined ""CBS This Morning"" the da...",2021-03-08 14:04:06+00:00,0 days 00:05:39,"[Oprah Winfrey, CBS This Morning, prince harry...",News & Politics,14725,3387,4883.0,1834547,HD,CBS This Morning,"Each weekday morning, Gayle King, Anthony Maso...",2013-05-23 10:59:52+00:00,1880000,1455944229,29384,True,True
Tl9KT9RwiGc,UC-SJ6nODDmufqBzPBwCvYvQ,Harry and Meghan on how race factored into the...,"Prince Harry and Meghan, Duchess of Sussex, co...",2021-03-08 14:00:03+00:00,0 days 00:08:33,"[prince harry, Meghan Markle, duchess of susse...",News & Politics,12080,2825,4935.0,1276809,HD,CBS This Morning,"Each weekday morning, Gayle King, Anthony Maso...",2013-05-23 10:59:52+00:00,1880000,1455944229,29384,True,True
Wx4Vpm1KzSY,UCKjU3KzdbJE1EFcHVqXC3_g,"Prince Harry, Meghan reveal struggles behind r...",Prince Harry and Meghan Markle's revealing int...,2021-03-08 05:18:20+00:00,0 days 00:04:46,"[harry and meghan, prince harry and meghan mar...",News & Politics,32428,10771,NaN,4112536,HD,CBC News: The National,The National is the flagship news and current ...,2007-10-17 21:34:25+00:00,996000,531856230,17987,True,True
mbwMspaiUVg,UCiWLfSweyRNmLpgEHekhoAg,First Take reacts to Blake Griffin to the Nets,First Take reacts to Blake Griffin to the Nets...,2021-03-08 17:07:45+00:00,0 days 00:07:08,"[espn, first take, blake griffin, nba, nba on ...",Sports,16410,424,3321.0,1104602,HD,ESPN,ESPN on YouTube features up-to-the-minute spor...,2005-10-31 23:34:22+00:00,7730000,7099162935,37990,True,True


## Stats

,length,stats.like,stats.dislike,stats.comment,stats.view,creator.stats.follower,creator.stats.view,creator.stats.post
count,296,2.960000e+02,296.000000,283.000000,2.960000e+02,2.960000e+02,2.960000e+02,296.000000
mean,0 days 00:10:38.929054054,1.402340e+05,4499.597973,13747.660777,2.807984e+06,5.585276e+06,1.806331e+09,5951.618243
std,0 days 00:09:42.428287606,2.302619e+05,16553.044548,28958.338820,3.804055e+06,7.743772e+06,2.549494e+09,11698.612058
min,0 days 00:00:14,3.260000e+02,128.000000,339.000000,9.052100e+04,1.100000e+04,1.220866e+07,18.000000
25%,0 days 00:03:22,1.456600e+04,397.000000,2381.000000,7.430280e+05,9.975000e+05,2.863848e+08,232.000000
50%,0 days 00:08:21,4.690500e+04,964.000000,4883.000000,1.411170e+06,1.940000e+06,8.009516e+08,745.000000
75%,0 days 00:13:05.250000,1.627430e+05,3830.750000,12036.000000,3.058373e+06,8.370000e+06,1.922072e+09,4762.000000
max,0 days 00:49:23,1.516146e+06,142630.000000,182949.000000,2.082677e+07,5.470000e+07,1.393113e+10,49574.000000


---